In [1]:
import re
import gensim
from gensim.parsing.preprocessing import strip_punctuation, strip_short, strip_numeric, strip_multiple_whitespaces, remove_stopwords
from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import urllib.request
import zipfile
import lzma
import shutil

In [2]:
import logging
logging.basicConfig(level=logging.INFO, force = True)
logger = logging.getLogger()
logger.info("Logging initialized")

INFO:root:Logging initialized


In [3]:
#Link found here: https://metatext.io/datasets/cc100-belarusian
urllib.request.urlretrieve('https://data.statmt.org/cc-100/be.txt.xz', 
                           'be.txt.xz')

urllib.request.urlretrieve('https://github.com/Belarus/GrammarDB/archive/refs/tags/PUBLICATION_2021.zip', 
                           'GrammarDB.zip')

('GrammarDB.zip', <http.client.HTTPMessage at 0x174669610>)

In [4]:
with lzma.open("be.txt.xz", "rb") as fsrc:
    with open("be.txt", "wb") as fdst:
        shutil.copyfileobj(fsrc, fdst)

with zipfile.ZipFile('GrammarDB.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [5]:
import xml.dom.minidom
from itertools import islice

def calculate_mapping_from_forms_to_base(filepath):
    xml_doc = xml.dom.minidom.parse(filepath)
    words = xml_doc.getElementsByTagName('Variant')
    result = {}
    collision_count = 0
    collisions = set()
    for word in words:
        base = word.getAttribute('lemma').replace("+", "").lower()
        forms_objs = word.getElementsByTagName('Form')
        local_map = {}
        citation_count = max([forms_obj.getAttribute('slouniki').count(',') for forms_obj in forms_objs]) + 1
        for forms_obj in forms_objs:
            if len(forms_obj.childNodes) > 0:
                form = forms_obj.childNodes[0].data.replace("+", "").lower()
                local_map[form] = (base, citation_count)
        for k, v in local_map.items():
            if k in result:
                if result[k][1] == v[1] and result[k][0] != v[0]:
                    collision_count += 1
                    collisions.add(v[0])
                    collisions.add(result[k][0])
                elif result[k][1] < v[1]:
                    result[k] = v
            else:
                result[k] = v
        #result.update(local_map)
    logger.info(f"Collisions (forms leading to different base word, and having same amount of citation): {collision_count}")
    logger.info(f"Examples of collisions: {list(islice(collisions, 5))}")
    for k in result:
        result[k] = result[k][0]
    return result

In [6]:
#verbs
v = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/V.xml')

#proper nouns
np = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/NP.xml')

#nouns
n1 = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/N1.xml')
n2 = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/N2.xml')
n3 = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/N3.xml')

#adjectives
adj1 = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/A1.xml')
adj2 = calculate_mapping_from_forms_to_base('GrammarDB-PUBLICATION_2021/A2.xml')

WORD_MAP = {}
WORD_MAP.update(v)
WORD_MAP.update(np)
WORD_MAP.update(n1)
WORD_MAP.update(n2)
WORD_MAP.update(n3)
WORD_MAP.update(adj1)
WORD_MAP.update(adj2)

INFO:root:Collisions (forms leading to different base word, and having same amount of citation): 2597
INFO:root:Examples of collisions: ['адапраць', 'навяліцца', 'адчыняць', 'мурчэць', 'спіцца']
INFO:root:Collisions (forms leading to different base word, and having same amount of citation): 139
INFO:root:Examples of collisions: ['уладзімір', 'барыс', 'захар', 'маркаў', 'павел']
INFO:root:Collisions (forms leading to different base word, and having same amount of citation): 1345
INFO:root:Examples of collisions: ['гайка', 'дамка', 'ёта', 'важычаня', 'зажор']
INFO:root:Collisions (forms leading to different base word, and having same amount of citation): 1155
INFO:root:Examples of collisions: ['кум', 'лісце', 'кош', 'лёс', 'нома']
INFO:root:Collisions (forms leading to different base word, and having same amount of citation): 954
INFO:root:Examples of collisions: ['ростра', 'удава', 'тэхніка', 'трух', 'руно']
INFO:root:Collisions (forms leading to different base word, and having same amo

In [7]:
print(len(WORD_MAP))

2282675


In [8]:
print(WORD_MAP['рухам'])
print(WORD_MAP['беларусі'])

рух
беларусь


In [9]:
def strip_trailing_newline(iterable):
    for i in iterable:
        yield i.rstrip()

CHARACTERS_MAP = {'ý': 'ў', 'i': 'і', 'ньн': 'нн', 'цьц': 'цц', 'сьц': 'сц', 'сьл':'сл'}
def replace_characters(iterable):
    for s in iterable:
        for k, v in CHARACTERS_MAP.items():
            s = s.replace(k, v)
        yield s

def split_sentences(iterable):
    for i in iterable:
        merged_dots = re.sub("[\.]+", ".", i)
        sentences = merged_dots.split('.')
        for s in sentences:
            yield s

STOPWORDS = ['на', 'не', 'што', 'да', 'па', 'як', 'за', 'для', 'гэта', 'ад', 'але', 'калі', 'пра', 'у', 'яго', 'якія', 'ён', 'іх', 'мы', 'каб', 'толькі', 'аб', 'таксама', 'які', 'ці', 'быў', 'было', 'яны', 'так', 'вы', 'яе', 'будзе', 'пры', 'яшчэ', 'тым', 'таму', 'вельмі', 'былі', 'можна', 'яна', 'пасля', 'пад', 'можа', 'дзе', 'якая', 'тут', 'была', 'трэба', 'тое', 'таго', 'або', 'гэтым', 'бо', 'ўсё', 'хто', 'ня', 'нас', 'гэтага', 'быць', 'гэты', 'ёсць', 'праз', 'ўжо', 'са', 'нават', 'то', 'мяне', 'ім','ва', 'той', 'усе', 'без', 'чым', 'мне', 'мае', 'сябе', 'гэтай', 'там', 'усё', 'вось', 'ды', 'каля', 'якіх', 'ты', 'якой', 'ўсе', 'жа', 'ужо', 'паводле', 'будуць', 'аднак', 'могуць', 'сваю', 'ні', 'сваёй', 'яму', 'свае', 'гэтыя', 'проста', 'ўсіх', 'якім', 'падчас', 'тады', 'свой', 'вас', 'паміж', 'нам', 'раз', 'сваіх', 'нашай', 'менавіта', 'перад', 'вам', 'тых','зь','такім', 'свайго', 'над', 'зараз', 'амаль', 'чаму', 'ёй', 'чынам', 'напрыклад', 'якога', 'якое', 'сваім', 'можаце', 'га', 'хоць', 'бы', 'тыя', 'такія', 'потым', 'адным', 'такі', 'якую', 'сабе','сам','гэтых','мая','наш','зусім','чаго','наша','зноў','дык','такіх','нашага','адразу','каго','самі','ст','ну','нашы','нашым','самы','яно','гэтае','дзеля','куды','by','гг']
def preprocess_sentences(iterable):
    for i in iterable:
        s = strip_multiple_whitespaces(strip_numeric(strip_short(strip_punctuation(i))))
        s = s.lower()
        s = re.sub("[«»“”„…—°′²]", "", s)
        s = remove_stopwords(s, stopwords=STOPWORDS)
        s = ' '.join([WORD_MAP.get(w, w) for w in s.split()])
        yield s

def remove_short_lines(iterable):
    for i in iterable:
        if not i.isspace() and len(i) >= 20:
            yield i


In [10]:
with open('be.txt', 'r') as original_file:
    with open('sentences.txt', 'w') as sentences_file:
        for s in remove_short_lines(preprocess_sentences(split_sentences(replace_characters(strip_trailing_newline(original_file))))):
            sentences_file.write(s + "\n")

In [11]:
loss_list = []
class Callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_list.append(loss)
        print('Loss after epoch {}:{}'.format(self.epoch, loss))
        model.running_training_loss = 0.0
        self.epoch = self.epoch + 1

In [12]:
model = Word2Vec(vector_size=100, window=3, min_count=10, workers=5)

INFO:gensim.utils:Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.025>', 'datetime': '2023-04-23T10:21:24.381682', 'gensim': '4.3.1', 'python': '3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'event': 'created'}


In [13]:
sentences = LineSentence('sentences.txt')

In [14]:
model.build_vocab(sentences, progress_per=5000000)

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #5000000, processed 44718951 words, keeping 810955 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000000, processed 89510301 words, keeping 1244638 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #15000000, processed 134174824 words, keeping 1580646 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000000, processed 178959997 words, keeping 1869907 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #25000000, processed 223611015 words, keeping 2135119 word types
INFO:gensim.models.word2vec:collected 2151510 word types from a corpus of 226598275 raw words and 25339681 sentences
INFO:gensim.models.word2vec:Creating a fresh vocabulary
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 277148 unique

In [15]:
model.train(sentences, epochs=100, total_examples=model.corpus_count, total_words=model.corpus_total_words, compute_loss=True, report_delay=180, callbacks=[Callback()])


INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'training model with 5 workers on 277148 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=3 shrink_windows=True', 'datetime': '2023-04-23T10:22:46.421779', 'gensim': '4.3.1', 'python': '3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'event': 'train'}
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 0.59% examples, 1285232 words/s, in_qsize 10, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 0: training on 226598275 raw words (219974027 effective words) took 168.2s, 1307777 effective words/s


Loss after epoch 0:45621836.0


INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 0.57% examples, 1239446 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1: training on 226598275 raw words (219973293 effective words) took 165.8s, 1326665 effective words/s


Loss after epoch 1:44355904.0


INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 0.56% examples, 1229852 words/s, in_qsize 8, out_qsize 2
INFO:gensim.models.word2vec:EPOCH 2: training on 226598275 raw words (219970716 effective words) took 169.2s, 1299881 effective words/s


...

INFO:gensim.models.word2vec:EPOCH 98 - PROGRESS: at 0.58% examples, 1281189 words/s, in_qsize 10, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 98: training on 226598275 raw words (219973683 effective words) took 168.8s, 1303259 effective words/s


Loss after epoch 98:38193532.0


INFO:gensim.models.word2vec:EPOCH 99 - PROGRESS: at 0.58% examples, 1280493 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 99: training on 226598275 raw words (219974942 effective words) took 168.1s, 1308280 effective words/s
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'training on 22659827500 raw words (21997472888 effective words) took 16488.3s, 1334127 effective words/s', 'datetime': '2023-04-23T15:48:08.114033', 'gensim': '4.3.1', 'python': '3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'event': 'train'}


Loss after epoch 99:37834140.0


(21997472888, 22659827500)

In [ ]:
model.save("word2vec-100-bel-cc100.model")

In [25]:
model.wv.most_similar('кіт', topn=20)

[('кастаненка', 0.6006743907928467),
 ('клешчукевіч', 0.5838990211486816),
 ('беляжэнкі', 0.5773590803146362),
 ('крэпак', 0.5444377660751343),
 ('чарнаглаз', 0.5400937795639038),
 ('патон', 0.5291008353233337),
 ('звоскага', 0.5288887619972229),
 ('вальеха', 0.5172730684280396),
 ('святловым', 0.5152472853660583),
 ('бачкоўскі', 0.511944591999054),
 ('нікульшын', 0.5117442011833191),
 ('бокшы', 0.5115799903869629),
 ('грабеншчыкоў', 0.506933331489563),
 ('герстэн', 0.5058147311210632),
 ('заборава', 0.4989950656890869),
 ('эрына', 0.4967189133167267),
 ('рагуля', 0.4914279878139496),
 ('ласкорын', 0.4895833432674408),
 ('грабеншчыкова', 0.48926597833633423),
 ('парафянюк', 0.48552122712135315)]

In [20]:
model.wv.index_to_key

['год',
 'беларускі',
 'беларусь',
 'чалавек',
 'час',
 'дзень',
 'большыць',
 'мова',
 'новы',
 'вялікі',
 'краіна',
 'жыццё',
 'праца',
 'горад',
 'месца',
 'справа',
 'гісторыя',
 'рэспубліка',
 'слова',
 'кожны',
 'гульня',
 'дзіцё',
 'дзяржаўны',
 'сайт',
 'працаваць',
 'нацыянальны',
 'раён',
 'пытанне',
 'імя',
 'казаць',
 'галоўны',
 'свет',
 'культура',
 'кніга',
 'старонка',
 'мець',
 'раз',
 'другі',
 'атрымаць',
 'розны',
 'адзін',
 'арганізацыя',
 'школа',
 'апошні',
 'адбыцца',
 'ведаць',
 'дом',
 'зрабіць',
 'мінск',
 'цэнтр',
 'матэрыял',
 'вынік',
 'хацець',
 'правіць',
 'рабіць',
 'праект',
 'цяпер',
 'беларус',
 'права',
 'адукацыя',
 'вайна',
 'частка',
 'праграма',
 'жыць',
 'дзяржава',
 'зямля',
 'шмат',
 'міжнародны',
 'сёння',
 'гад',
 'работа',
 'інфармацыя',
 'развіццё',
 'выбар',
 'няма',
 'лепшы',
 'народны',
 'яўляцца',
 'наступны',
 'палітычны',
 'іншых',
 'праблема',
 'пачатак',
 'вёска',
 'народ',
 'магчы',
 'аляксандр',
 'кіраўнік',
 'знаходзіцца',
 'мя

In [ ]:
model.wv.get_vecattr("прыдумляць", "count")

In [26]:
loss_list

[45621836.0,
 44355904.0,
 44249936.0,
 44104492.0,
 44107028.0,
 44097436.0,
 44270596.0,
 44089604.0,
 44185456.0,
 44285424.0,
 44182396.0,
 44126032.0,
 44408908.0,
 44199988.0,
 44425624.0,
 44209516.0,
 44227556.0,
 44041368.0,
 44373620.0,
 44456688.0,
 44070200.0,
 43995008.0,
 44134528.0,
 44107632.0,
 43802268.0,
 44047244.0,
 43904480.0,
 43939264.0,
 43987436.0,
 43859148.0,
 43961468.0,
 44073332.0,
 43907992.0,
 43725336.0,
 43849884.0,
 43924944.0,
 43826232.0,
 43496588.0,
 43605268.0,
 43641544.0,
 43543980.0,
 43537204.0,
 43491140.0,
 43373268.0,
 43624628.0,
 43330720.0,
 43449924.0,
 43234040.0,
 43097712.0,
 44565932.0,
 43416728.0,
 43234072.0,
 43260536.0,
 43074152.0,
 43096868.0,
 42714596.0,
 43268708.0,
 42984456.0,
 43917916.0,
 43035192.0,
 42818392.0,
 42656488.0,
 42411144.0,
 42707748.0,
 42523348.0,
 42491780.0,
 42605616.0,
 42294312.0,
 42310540.0,
 42306532.0,
 42327836.0,
 42028380.0,
 43780020.0,
 41960848.0,
 41753348.0,
 41644528.0,
 42072300.0,

In [29]:
WORD_MAP['сабака']

'сабака'